In [86]:
import pandas as pd
import requests
import os

In [3]:
df = pd.read_pickle('/kaggle/input/semantic-scholar-retrieval/retrieved_data')

In [51]:
def extract_pdf_from_response(df_row, pdf_directory='/kaggle/working/pdf_directory'):
    """
    This function takes the structure of the 'openAccessPdf' column of the semantic scholar
    api response dataset downloads the available pdfs in the specified directory, indexed
    with the paperid value.
    
    variables:
    - df_row: And iterable with a 'paperId' key and an 'openAccessPdf' key
    - pdf_directory: The folder in which the files will be stored
    """
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    if df_row['openAccessPdf']:
        try:
            req = requests.get(df_row['openAccessPdf']['url'])
            if req.status_code == 200:
                with open(pdf_directory + '/' + df_row['paperId'] + '.pdf', 'w') as f:
                    f.write(req.text)
        except Exception as e:
            pass

In [52]:
df[~df['openAccessPdf'].isna()].head(5).apply(extract_pdf_from_response, axis=1)

http://journal.hmjournals.com/index.php/IJRISE/article/download/2972/2568/5442
http://www.cs.wpi.edu/~heineman/PDF/ICSE-2002.pdf
https://ejnmmiphys.springeropen.com/counter/pdf/10.1186/s40658-023-00600-4
HTTPConnectionPool(host='reo.project.cwi.nl', port=80): Max retries exceeded with url: /reo/export/2469/docs/QIAintoMC/related/e1093.pdf (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7cce54b48d60>: Failed to resolve 'reo.project.cwi.nl' ([Errno -2] Name or service not known)"))
https://dl.acm.org/doi/pdf/10.3115/1075527.1075593


2    None
0    None
2    None
0    None
0    None
dtype: object